# Import Libraries

In [1]:
import pandas as pd

from utils import clean_text, remove_numbers, remove_punctuations, remove_strings, remove_stopwords
from evaluation import multi_column_embedding_model_evaluation
from constants import (
    TEST_DATA_PATH,
    CLEANED_TEST_DATA_PATH,
    E5_LARGE_CONFIG_PATH,
    E5_LARGE_INSTRUCT_CONFIG_PATH,
    QWEN3_EMBEDDING_CONFIG_PATH,
    PARAPHRASER_EMBEDDING_CONFIG_PATH,
)

c:\Users\mk255155\AppData\Local\anaconda3\envs\velesenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mk255155\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Prepare Data

## Load Data

In [2]:
df_test = pd.read_csv(".." / TEST_DATA_PATH)
df_test.head(5)

,Item_Name,class,Brand,Weight,Number of units,Size of units,Price,T.Price,Pack,Unit
0,Americana Okra zero 400 gm,Vegetables & Fruits,Americana,400gm,1,NaN,NaN,NaN,كيس,gm
1,ليمون اداليا 500 جم,"Tins, Jars & Packets",NaN,500جم,1,NaN,NaN,NaN,علبة,جم
2,صلصه هاينز برطمان خصم عرض,"Tins, Jars & Packets",هاينز,NaN,1,NaN,NaN,NaN,علبة,NaN
3,Dasani water 330ML,Water,Dasani,330مل,1,NaN,NaN,NaN,زجاجة,مل
4,بودرة عصير أناناس من سورس، 900 جم,Soft Drinks & Juices,سورس,900جم,1,NaN,NaN,NaN,عبوة,جم


In [3]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4773 entries, 0 to 4772
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Item_Name        4773 non-null   object 
 1   class            4772 non-null   object 
 2   Brand            3827 non-null   object 
 3   Weight           3022 non-null   object 
 4   Number of units  4773 non-null   int64  
 5   Size of units    45 non-null     object 
 6   Price            278 non-null    float64
 7   T.Price          278 non-null    float64
 8   Pack             4604 non-null   object 
 9   Unit             3058 non-null   object 
dtypes: float64(2), int64(1), object(7)
memory usage: 373.0+ KB


## Remove Nulls

In [4]:
df_test.dropna(subset=["Item_Name", "class"], inplace=True)
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4772 entries, 0 to 4772
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Item_Name        4772 non-null   object 
 1   class            4772 non-null   object 
 2   Brand            3826 non-null   object 
 3   Weight           3021 non-null   object 
 4   Number of units  4772 non-null   int64  
 5   Size of units    45 non-null     object 
 6   Price            278 non-null    float64
 7   T.Price          278 non-null    float64
 8   Pack             4603 non-null   object 
 9   Unit             3057 non-null   object 
dtypes: float64(2), int64(1), object(7)
memory usage: 410.1+ KB


## Normalize Data

In [5]:
df_test["Item_Name"] = df_test["Item_Name"].str.lower()
df_test["class"] = df_test["class"].str.lower()
df_test["Unit"] = df_test["Unit"].str.lower()
df_test["Weight"] = df_test["Weight"].str.lower()
df_test["Brand"] = df_test["Brand"].str.lower()
df_test.head()

,Item_Name,class,Brand,Weight,Number of units,Size of units,Price,T.Price,Pack,Unit
0,americana okra zero 400 gm,vegetables & fruits,americana,400gm,1,NaN,NaN,NaN,كيس,gm
1,ليمون اداليا 500 جم,"tins, jars & packets",NaN,500جم,1,NaN,NaN,NaN,علبة,جم
2,صلصه هاينز برطمان خصم عرض,"tins, jars & packets",هاينز,NaN,1,NaN,NaN,NaN,علبة,NaN
3,dasani water 330ml,water,dasani,330مل,1,NaN,NaN,NaN,زجاجة,مل
4,بودرة عصير أناناس من سورس، 900 جم,soft drinks & juices,سورس,900جم,1,NaN,NaN,NaN,عبوة,جم


## Clean Data

In [6]:
df_test["removed_punctuations"] = df_test["Item_Name"].apply(remove_punctuations)
df_test["removed_numbers"] = df_test["Item_Name"].apply(lambda x: remove_numbers(x, remove_string=False))
df_test["removed_brand"] = df_test.apply(lambda x: remove_strings(x.Item_Name, [x.Brand]), axis=1)
df_test["removed_pack"] = df_test.apply(lambda x: remove_strings(x.Item_Name, [x.Pack]), axis=1)
df_test["removed_unit"] = df_test.apply(lambda x: remove_strings(x.Item_Name, [x.Unit]), axis=1)
df_test["removed_numbers_and_punctuations"] = df_test["Item_Name"].apply(lambda x: remove_numbers(remove_punctuations(x), remove_string=False))
df_test["removed_stopwords"] = df_test["Item_Name"].apply(remove_stopwords)
df_test["cleaned_text"] = df_test.apply(clean_text, axis=1)
df_test

,Item_Name,class,Brand,Weight,Number of units,Size of units,Price,T.Price,Pack,Unit,removed_punctuations,removed_numbers,removed_brand,removed_pack,removed_unit,removed_numbers_and_punctuations,removed_stopwords,cleaned_text
0,americana okra zero 400 gm,vegetables & fruits,americana,400gm,1,NaN,NaN,NaN,كيس,gm,americana okra zero 400 gm,americana okra zero gm,okra zero 400 gm,americana okra zero 400 gm,americana okra zero 400,americana okra zero gm,americana okra zero 400 gm,okra zero gm
1,ليمون اداليا 500 جم,"tins, jars & packets",NaN,500جم,1,NaN,NaN,NaN,علبة,جم,ليمون اداليا 500 جم,ليمون اداليا جم,ليمون اداليا 500 جم,ليمون اداليا 500 جم,ليمون اداليا 500,ليمون اداليا جم,ليمون اداليا 500 جم,ليمون اداليا جم
2,صلصه هاينز برطمان خصم عرض,"tins, jars & packets",هاينز,NaN,1,NaN,NaN,NaN,علبة,NaN,صلصه هاينز برطمان خصم عرض,صلصه هاينز برطمان خصم عرض,صلصه برطمان خصم عرض,صلصه هاينز برطمان خصم عرض,صلصه هاينز برطمان خصم عرض,صلصه هاينز برطمان خصم عرض,صلصه هاينز برطمان خصم عرض,صلصه برطمان خصم عرض
3,dasani water 330ml,water,dasani,330مل,1,NaN,NaN,NaN,زجاجة,مل,dasani water 330ml,dasani water ml,water 330ml,dasani water 330ml,dasani water 330ml,dasani water ml,dasani water 330ml,water ml
4,بودرة عصير أناناس من سورس، 900 جم,soft drinks & juices,سورس,900جم,1,NaN,NaN,NaN,عبوة,جم,بودرة عصير أناناس من سورس 900 جم,بودرة عصير أناناس من سورس، جم,بودرة عصير أناناس من ، 900 جم,بودرة عصير أناناس من سورس، 900 جم,بودرة عصير أناناس من سورس، 900,بودرة عصير أناناس من سورس جم,بودرة عصير أناناس سورس، 900 جم,بودرة عصير أناناس جم
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4768,جهينه كوكتيل 1ل,soft drinks & juices,جهينه,1ل,1,NaN,NaN,NaN,زجاجة,ل,جهينه كوكتيل 1ل,جهينه كوكتيل ل,كوكتيل 1ل,جهينه كوكتيل 1ل,جهينه كوكتي 1,جهينه كوكتيل ل,جهينه كوكتيل 1ل,كوكتيل
4769,دبل ديرى مشروب بنكهه ليمون 275مل,soft drinks & juices,دبل ديرى,275مل,1,NaN,NaN,NaN,زجاجة,مل,دبل ديرى مشروب بنكهه ليمون 275مل,دبل ديرى مشروب بنكهه ليمون مل,مشروب بنكهه ليمون 275مل,دبل ديرى مشروب بنكهه ليمون 275مل,دبل ديرى مشروب بنكهه ليمون 275,دبل ديرى مشروب بنكهه ليمون مل,دبل ديرى مشروب بنكهه ليمون 275مل,مشروب بنكهه ليمون مل
4770,galaxy smooth milk chocolate 24 x 36g,"chocolates, sweets & desserts",galaxy,36جم,24,NaN,NaN,NaN,عبوة,جم,galaxy smooth milk chocolate 24 x 36g,galaxy smooth milk chocolate x g,smooth milk chocolate 24 x 36g,galaxy smooth milk chocolate 24 x 36g,galaxy smooth milk chocolate 24 x 36g,galaxy smooth milk chocolate x g,galaxy smooth milk chocolate 24 x 36g,smooth milk chocolate x g
4771,hot ketchup squeeze – 320g,"sauces, dressings & condiments",NaN,320g,1,NaN,NaN,NaN,عبوة,g,hot ketchup squeeze 320g,hot ketchup squeeze – g,hot ketchup squeeze – 320g,hot ketchup squeeze – 320g,hot ketchup squeeze – 320,hot ketchup squeeze g,hot ketchup squeeze – 320g,hot ketchup squeeze g


In [7]:
df_test[df_test["cleaned_text"]==""]

,Item_Name,class,Brand,Weight,Number of units,Size of units,Price,T.Price,Pack,Unit,removed_punctuations,removed_numbers,removed_brand,removed_pack,removed_unit,removed_numbers_and_punctuations,removed_stopwords,cleaned_text


In [8]:
df_test = df_test[~(df_test["cleaned_text"]=="")]

# Evaluate Model

In [9]:
n_sample = None

In [10]:

multi_column_embedding_model_evaluation(
    df_test,
    ".." / E5_LARGE_INSTRUCT_CONFIG_PATH, 
    [
        # "Item_Name", 
        # "removed_stopwords",
        "cleaned_text",
    ],
    n_sample
)

c:\Users\mk255155\AppData\Local\anaconda3\envs\velesenv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mk255155\.cache\huggingface\hub\models--intfloat--multilingual-e5-large-instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


KeyboardInterrupt: 

In [ ]:
multi_column_embedding_model_evaluation(
    df_test, 
    ".." / QWEN3_EMBEDDING_CONFIG_PATH,
    [
        # "Item_Name", 
        # "removed_punctuations", 
        # "removed_numbers", 
        # "removed_numbers_and_punctuations", 
        # "removed_brand", 
        # "removed_pack", 
        # "removed_unit", 
        # "removed_stopwords",
        "cleaned_text"
    ],
    n_sample
)

Average time taken for a single example: 0.1553166389465332 seconds
Number of examples: 5
The score of using cleaned_text: 0.4


In [ ]:

multi_column_embedding_model_evaluation(
    df_test,
    ".." / PARAPHRASER_EMBEDDING_CONFIG_PATH,
    [
        # "Item_Name", 
        # "removed_punctuations", 
        # "removed_numbers", 
        # "removed_numbers_and_punctuations", 
        # "removed_brand", 
        # "removed_pack", 
        # "removed_unit", 
        "cleaned_text"
    ], 
    n_sample
)

Average time taken for a single example: 0.021748514856896477 seconds
Number of examples: 4772
The score of using cleaned_text: 0.3265305324526368


# Save Cleaned Data

In [11]:
df_test.drop_duplicates(subset=["Item_Name"], inplace=True)
df_test.drop(columns=["Brand", "Weight", "Number of units", "Size of units", "Price", "T.Price", "Pack", "Unit"], inplace=True)

In [12]:
df_test = df_test[~(df_test["cleaned_item_name"]=="")]

KeyError: 'cleaned_item_name'

In [13]:
df_test.to_csv(".." / CLEANED_TEST_DATA_PATH, index=False, encoding="utf-8-sig")